In [ ]:
import pandas as pd  
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD, evaluate
from konlpy.tag import Twitter

ModuleNotFoundError: No module named 'surprise'

In [9]:
original_data = pd.read_csv('./eyeShadow.csv')
type = {'건성': 0, '지성': 1, '중성': 2, '복합성': 3, '민감성': 4}
original_data['type'] = original_data['type'].map(type)

In [42]:
twitter = Twitter()
totalWord = ''
for index, data in enumerate(original_data['review']):
    words = twitter.pos(data, norm=True)
    for word in words:
        if word[1] not in ["Punctuation", "Eomi", "Josa"]:
            totalWord += word[0] + ' '
    original_data.loc[index, 'review'] = totalWord
    totalWord = ''

,userId,name,popId,rate,review,type
0,0,매트 아이 컬러,0,4.57,진저 진짜 짱 추천 합니다 코섀디 완전 자연스럽고 그냥 단독 진짜 이뻐요 웜톤 특히...,0
1,1,매트 아이 컬러,0,4.57,진저 평생 이 것 쓸거에요,0
2,2,매트 아이 컬러,0,4.57,완전 매트 다 제 좋아하는 색 이 너무 좋아요,0


In [62]:
original_data.head(3)

,userId,name,popId,rate,review,type
0,0,매트 아이 컬러,0,4.57,진저 진짜 짱 추천 합니다 코섀디 완전 자연스럽고 그냥 단독 진짜 이뻐요 웜톤 특히...,0
1,1,매트 아이 컬러,0,4.57,진저 평생 이 것 쓸거에요,0
2,2,매트 아이 컬러,0,4.57,완전 매트 다 제 좋아하는 색 이 너무 좋아요,0


In [63]:
id_purify_data = original_data.copy()
id_purify_data.head(3)

,userId,name,popId,rate,review,type
0,0,매트 아이 컬러,0,4.57,진저 진짜 짱 추천 합니다 코섀디 완전 자연스럽고 그냥 단독 진짜 이뻐요 웜톤 특히...,0
1,1,매트 아이 컬러,0,4.57,진저 평생 이 것 쓸거에요,0
2,2,매트 아이 컬러,0,4.57,완전 매트 다 제 좋아하는 색 이 너무 좋아요,0


In [64]:
member_per_type = {0: 864, 1: 849, 2: 775, 3: 855, 4: 835}
for i in range(5):
    np.random.seed(42)
    id_purify_data.loc[id_purify_data["type"] == i, "userId"] = [np.random.randint(i * 200, 200 * (i + 1))
                                                                 for j in range(0, member_per_type[i])]

In [65]:
id_purify_data.head(5)

,userId,name,popId,rate,review,type
0,102,매트 아이 컬러,0,4.57,진저 진짜 짱 추천 합니다 코섀디 완전 자연스럽고 그냥 단독 진짜 이뻐요 웜톤 특히...,0
1,179,매트 아이 컬러,0,4.57,진저 평생 이 것 쓸거에요,0
2,92,매트 아이 컬러,0,4.57,완전 매트 다 제 좋아하는 색 이 너무 좋아요,0
3,14,매트 아이 컬러,0,4.57,좋은 거 하나 사서 힛팬 하는 걸 목표 프레스코 구매 함 프레스코 저렴 많이 가지 ...,0
4,106,매트 아이 컬러,0,4.57,한창 섀도우 빠져 있을 때 뭐 살 고민 하다가 평이 좋아서 진저 구매 해서 사용 해...,0


In [66]:
id_purify_data = id_purify_data.sort_values('userId')

In [67]:
id_purify_data.head(3)

,userId,name,popId,rate,review,type
3453,0,제미네이트 아이즈,78,4.57,루나 솔 쉐도우 손 많이 갑니다 \r\n\r\n 봄 여름 코랄 코랄 많이 쓰는데 \...,0
2629,0,힛팬 섀도우,59,4.22,밀크 티 라떼 기본 음영 컬러 임 \r\n 제 형 가볍고 부드러워서 발 색도 잘 되...,0
2403,0,플레이 컬러 아이즈 와인파티,54,4.34,에뛰드 기본 적 색조 화장품 정말 좋은 듯 \r\n 주로 버건디 브라운 조합 화장 ...,0


In [68]:
def reviewData(type):
    review_data = ['' for i in range(100)]
    review_type_popid_data = pd.DataFrame(id_purify_data[["popId", "type", "review"]])
    review_type_popid_data = review_type_popid_data.reset_index(drop=True)
    review_type_popid_data = review_type_popid_data.loc[review_type_popid_data["type"] == type, :]
    for index, row in review_type_popid_data.iterrows():
        review_data[row["popId"]] += row["review"]
    return review_data

In [73]:
review_data = reviewData(1)

In [74]:
id_to_name = original_data[["popId", "name"]].drop_duplicates()
id_to_name = id_to_name.reset_index(drop=True)
id_to_name = id_to_name.drop(columns="popId", axis=1)

In [75]:
name_to_id = original_data[["popId", "name"]].drop_duplicates()
name_to_id = name_to_id.set_index("name", drop=True)

In [76]:
review_data = pd.Series(review_data)
tf = TfidfVectorizer(analyzer='word', min_df=2, stop_words=['\r', '\n'], sublinear_tf=True)
tf_matrix = tf.fit_transform(review_data)
cosine_sim = linear_kernel(tf_matrix, tf_matrix)

In [77]:
def making_evaluate_data(type = 0):
    evaluate_data = id_purify_data[id_purify_data["type"]==type]
    evaluate_data = evaluate_data.drop(columns=['name', 'review', 'type'], axis=1)
    print(evaluate_data)
    reader = Reader()
    evaluate_data = Dataset.load_from_df(evaluate_data, reader)
    return evaluate_data

In [56]:
evaluate_data = making_evaluate_data(0)

      userId  popId  rate
3453       0     78  4.57
2629       0     59  4.22
2403       0     54  4.34
3600       0     82  4.30
668        0     15  4.53
3996       0     91  3.97
91         1      2  4.61
1071       1     24  4.88
2665       1     60  4.27
1374       1     31  4.41
357        1      8  4.47
3950       2     90  4.15
3604       2     82  4.30
4085       2     93  4.29
1198       2     27  4.33
896        2     20  4.39
4035       3     92  4.55
2631       3     59  4.22
361        3      8  4.47
1730       3     39  4.08
669        4     15  4.53
3815       4     87  4.35
3276       4     74  4.21
576        4     13  4.46
2223       4     50  4.44
1775       5     40  4.18
2226       5     50  4.44
3945       5     90  4.15
3234       6     73  4.30
1332       6     30  4.42
...      ...    ...   ...
4175     191     95  4.43
4037     192     92  4.55
628      192     14  5.00
4124     192     94  4.16
1547     193     35  4.29
3810     193     87  4.35
1949     193

In [78]:
evaluate_data.split(5)
svd = SVD()
evaluate(svd, evaluate_data, measures=['RMSE', 'MAE'])

C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.1410
MAE:  0.1132
------------
Fold 2
RMSE: 0.1476
MAE:  0.1160
------------
Fold 3
RMSE: 0.1346
MAE:  0.1032
------------
Fold 4
RMSE: 0.1464
MAE:  0.1177
------------
Fold 5
RMSE: 0.1452
MAE:  0.1123
------------
------------
Mean RMSE: 0.1430
Mean MAE : 0.1125
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.1409628940320353,
                             0.14763233298228887,
                             0.13462245538454692,
                             0.14642515188787403,
                             0.14515908061967603],
                            'mae': [0.11323904693001625,
                             0.11600083735766148,
                             0.10320666017048018,
                             0.11770960484138365,
                             0.11228333726324301]})

In [79]:
from sklearn.externals import joblib

trainset = evaluate_data.build_full_trainset()
svd.fit(trainset)
joblib.dump(svd, 'eyeShadow.pkl')

['eyeShadow_1.pkl']

In [80]:
def making_predict_data(cosmetic_id):
    predict_data = original_data[['popId', 'name']]
    predict_data = predict_data.drop_duplicates()
    return predict_data.iloc[cosmetic_id]

In [81]:
def get_recommaned_cosmetic(userId, cosmeticName='매트 아이 컬러'):
    cosmetic_id = name_to_id.loc[cosmeticName, :]
    sim_scores = list(enumerate(cosine_sim[int(cosmetic_id)]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    cosmetic_id = [i[0] for i in sim_scores]
    prediction = making_predict_data(cosmetic_id)
    
    prediction['est'] = prediction['popId'].apply(lambda x: svd.predict(userId, x).est)
    prediction = prediction.sort_values('est', ascending=False)
    return prediction

In [ ]:
get_recommaned_cosmetic(15, '네이키드 히트')

,popId,name,est
223,5,피스매칭 12구 섀도우 팔레트,4.605906
1069,24,리미티드 아이 아티스트 팔레트,4.587822
1327,30,미네랄 슈가 블렌드 아이즈,4.489714
90,2,피스 매칭 섀도우 매트,4.485759
1024,23,컬러 필터 섀도우 팔레트,4.453131
844,19,피스매칭 섀도우 4구 팔레트,4.441341
401,9,스위트 피치 아이 섀도우 컬렉션,4.428288
1769,40,잉크 피팅 섀도우,4.378910
711,16,섀이드 앤 섀도우,4.371010
2174,49,초콜릿 바 아이섀도우 컬렉션,4.370236
